In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import re

In [2]:
ss = pd.read_csv("outliers_fixed_4.csv")
ss = ss.drop(["Unnamed: 0"], 1)
ss.head()

,subject,task,sensor,start,stop,length,outliers,extended,continuous,fixed,final_start,final_stop,Start_dif,Stop_dif,Start_dif_1s,Start_dif_notes,Stop_dif_1s,Stop_dif_notes
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3,chunk_extend,11.4125,21.4125,0.0375,0.0375,0,NaN,0,NaN
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3,chunk_extend,11.3750,21.3750,0.0375,0.0375,0,NaN,0,NaN
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1,regular,65.9625,76.0125,0.6125,0.5625,0,NaN,0,NaN
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1,extended,66.5750,76.5750,0.6125,0.5625,0,NaN,0,NaN
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1,extended,15.0500,25.0500,0.2875,0.2875,0,NaN,0,NaN


In [3]:
#Generate actual finalp air start/stops
pair_start = []
pair_stop = []

for i, r in ss.iterrows():
    if r["sensor"] == "right_1" or r["sensor"] == "left_1":
        if r["Start_dif_notes"] == 1:
            pair_start.append(r["final_start"])
        elif r["Start_dif_notes"] == 2:
            pair_start.append(ss.loc[i+1, "final_start"])
        else:
            pair_start.append(min(r["final_start"], ss.loc[i+1, "final_start"]))

        if r["Stop_dif_notes"] == 1:
            pair_stop.append(r["final_stop"])
        elif r["Stop_dif_notes"] == 2:
            pair_stop.append(ss.loc[i+1, "final_stop"])
        else:
            pair_stop.append(max(r["final_stop"], ss.loc[i+1, "final_stop"]))

    else:
        if r["Start_dif_notes"] == 1:
            pair_start.append(ss.loc[i-1, "final_start"])
        elif r["Start_dif_notes"] == 2:
            pair_start.append(r["final_start"])
        else:
            pair_start.append(min(r["final_start"], ss.loc[i-1, "final_start"]))

        if r["Stop_dif_notes"] == 1:
            pair_stop.append(ss.loc[i-1, "final_stop"])
        elif r["Stop_dif_notes"] == 2:
            pair_stop.append(r["final_stop"])
        else:
            pair_stop.append(max(r["final_stop"], ss.loc[i-1, "final_stop"]))

ss["pair_start"] = pair_start
ss["pair_stop"] = pair_stop
ss.loc[724, "pair_stop"] = 38.55 #fixing one small outlier
ss.loc[725, "pair_stop"] = 38.55
ss.head()

,subject,task,sensor,start,stop,length,outliers,extended,continuous,fixed,final_start,final_stop,Start_dif,Stop_dif,Start_dif_1s,Start_dif_notes,Stop_dif_1s,Stop_dif_notes,pair_start,pair_stop
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3,chunk_extend,11.4125,21.4125,0.0375,0.0375,0,NaN,0,NaN,11.3750,21.4125
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3,chunk_extend,11.3750,21.3750,0.0375,0.0375,0,NaN,0,NaN,11.3750,21.4125
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1,regular,65.9625,76.0125,0.6125,0.5625,0,NaN,0,NaN,65.9625,76.5750
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1,extended,66.5750,76.5750,0.6125,0.5625,0,NaN,0,NaN,65.9625,76.5750
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1,extended,15.0500,25.0500,0.2875,0.2875,0,NaN,0,NaN,15.0500,25.3375


In [4]:
#ss.to_csv("pair_start_stop.csv")

In [5]:
#sequence name format: PDMotion_0001_Retest_FT_Left_#.csv
#columns: time, left_1 derivative, left_2 derivative


In [6]:
#simple start stops:
simple = ss.copy()
simple = simple.filter(["subject", "task", "sensor", "pair_start", "pair_stop"])
simple = simple.iloc[::2]
simple = simple.reset_index(drop=True)
simple["sensor"] = simple["sensor"].str[:-2]
simple.head()

,subject,task,sensor,pair_start,pair_stop
0,PDMotion_0020_Test,PS,right,11.3750,21.4125
1,PDMotion_0020_Test,PS,left,65.9625,76.5750
2,PDMotion_0020_Test,HM,right,15.0500,25.3375
3,PDMotion_0020_Test,HM,left,31.6875,41.7250
4,PDMotion_0020_Test,FT,right,10.0250,20.0500


In [7]:
#add labels to simple
labels = pd.read_csv("labels.csv")
for i, r in simple.iterrows():
    #subject id, test, task id, l/r
    subject_id = int(re.findall(r'\d+', r["subject"])[0])
    retest_bool = True if r["subject"].count("Retest") > 0 else False
    test = "rt" if retest_bool else "t"
    task = r["task"]
    side = "l" if r["sensor"] == "left" else "r"

    result = labels.loc[(labels["subject_id"].str.contains(str(subject_id)) & (labels["test"] == test) & (labels["side"] == side) & (labels["task_id"] == task)),["label", "reason"]].to_numpy()
    simple.loc[i,"label"] = result[0][0]
    simple.loc[i, "reason"] = result[0][1]


In [8]:
#Testing for simple labels
labels = pd.read_csv("labels.csv")
r = simple.iloc[0]
print(r)
subject_id = int(re.findall(r'\d+', r["subject"])[0])
retest_bool = True if r["subject"].count("Retest") > 0 else False
test = "rt" if retest_bool else "t"
task = r["task"]
print(task)
side = "l" if r["sensor"] == "left" else "r"
result = labels.loc[(labels["subject_id"].str.contains(str(subject_id)) & (labels["test"] == test) & (labels["side"] == side) & (labels["task_id"] == task)),["label", "reason"]].to_numpy()
result = labels.loc[(labels["subject_id"].str.contains(str(subject_id)) & (labels["test"] == test) & (labels["side"] == side) & (labels["task_id"] == task)),["label", "reason"]].to_numpy()
print(result[0][0])

subject       PDMotion_0020_Test
task                          PS
sensor                     right
pair_start                11.375
pair_stop                21.4125
label                        0.0
reason                       NaN
Name: 0, dtype: object
PS
0


In [12]:
simple.to_csv("simple_start_stop.csv")
#simple.head()

In [10]:
for i, r in simple.iterrows():
    series1 = pd.read_csv("series/"+r["subject"]+"_"+r["task"]+"_"+r["sensor"]+"_1.csv")
    series2 = pd.read_csv("series/"+r["subject"]+"_"+r["task"]+"_"+r["sensor"]+"_2.csv")
    range_start = r["pair_start"]
    range_stop = min(r["pair_stop"], series1["relative_time"].iloc[-1], series1["relative_time"].iloc[-2])
    current_start = range_start
    current_stop = current_start+5
    current_count = 1

    while current_stop <= range_stop:
        start_index1 = series1.index.get_loc(series1.index[series1['relative_time'] == current_start][0])
        start_index2 = series2.index.get_loc(series2.index[series2['relative_time'] == current_start][0])
        stop_index1 = series1.index.get_loc(series1.index[series1['relative_time'] == current_stop][0])
        stop_index2 = series2.index.get_loc(series2.index[series2['relative_time'] == current_stop][0])

        one = series1["derivative"].iloc[start_index1:stop_index1].to_numpy()
        two = series2["derivative"].iloc[start_index2:stop_index2].to_numpy()
        rt =  series1["relative_time"].iloc[start_index1:stop_index1].to_numpy()

        combine = pd.DataFrame({"relative_time": rt, "sensor_1": one, "sensor_2": two})
        combine.to_csv("sequences/"+r["subject"]+"_"+r["task"]+"_"+r["sensor"]+"_"+str(current_count)+".csv")
        
        current_start+=1
        current_stop+=1
        current_count+=1

    simple.loc[i,"count"] = current_count

In [66]:
"PDMotion_0020_Retest".count("Retest")

1

In [11]:
simple.head()

,subject,task,sensor,pair_start,pair_stop,label,reason,count
0,PDMotion_0020_Test,PS,right,11.3750,21.4125,0.0,NaN,7.0
1,PDMotion_0020_Test,PS,left,65.9625,76.5750,1.0,NaN,7.0
2,PDMotion_0020_Test,HM,right,15.0500,25.3375,1.0,a,7.0
3,PDMotion_0020_Test,HM,left,31.6875,41.7250,0.0,NaN,7.0
4,PDMotion_0020_Test,FT,right,10.0250,20.0500,0.0,NaN,7.0
